####  Скрипта для сбора пользователей по множеству групп. 



1) Два варианта исполнения - из текстового файла или по запросу

* Текстовый файл - каждая группа должна быть расположена на новой строке текстового документа. Что это - ссылка или group_id - не важно. 
* Запрос - обычный поисковый запрос при поиске групп ВКонтакте. Скрипт обрабатывает до 50 самых крупных групп и делает по ним выжимку.

2) Вводится поисковый запрос, который распространяется на все группы. 

3) Получаем сведенную табличку, избавленную от повторяющихся значений

4) У итоговой таблицы есть дополнительный столбец - название группы.

5) Для того, чтобы поменять способ сбора данных между поиском и текстовым файлом - нужно закомментить ненужный и раскоментить нужный вариант. 



In [80]:
%%time
import requests
import re
from time import sleep
from collections import defaultdict
import pandas as pd
from datetime import date
year = date.today().year
import warnings
warnings.filterwarnings('ignore')

token = '09306d741547dda7fc610874a071b003df1218442356f91d5025a678e683a9dc2f1a2a35fbf00cbeeacc6'
document_name = 'test.txt'



def get_by_txt(file):
    """Функция открывает текстовый файл, построчно обрабатывает данные.
    Если это число - оставляет как есть, если ссылка, то превращает её в group_id"""
    ids = open(file).read().split('\n')
    step = 0
    for group in ids:
        if group.isdigit(): ids[step] = group
        elif 'vk.com' in group: 
            ids[step] = requests.get('https://api.vk.com/method/utils.resolveScreenName', 
                 params = {'v':'5.75', 'screen_name':ids[step].split('com/')[1],
                           'access_token':token}).json()['response']['object_id']
        else:
            ids.remove(group)
        step+=1
        
    return list(filter(None, ids))



def get_by_query():
    """Можно менять поле sort. Подробности внизу"""
    query = input('Введите поисковый запрос: ')
    req = requests.get('https://api.vk.com/method/groups.search', 
            params = {'access_token':token, 
                      'v':'5.73', 'q':str(query), 'count':50, 'sort':0}).json()['response']
    
    groups = [req['items'][i]['id'] for i in range(len(req['items']))]
    return groups



def excel_writer(dataframe):
    """Функция для записи датафрейма в эксель"""
    while True:
        try:
            choice = str(input('Хотите сохранить в эксель?[Y/N] ').lower())
        except ValueError:
            print("Попробуйте ещё раз, с первого раза не пошло")
            #better try again... Return to the start of the loop
            continue
        else:
            break


    if choice in ['y', 'да', '1']:
        writer = pd.ExcelWriter('{}.xlsx'.format(str(input('Введите название файла: '))))
        dataframe.to_excel(writer,'Sheet1', index = False)
        writer.save()   
        print('Файл сохранен')    
    else:
        print('Работа программы завершена')
        


    
# owner_id = input()
class Parcer:
    def __init__(self, owner_id):
        self.owner_id = owner_id   
        self.get_members()
        self.preparation()


    def get_members(self):
        """Парсим группу, получаем словарь"""
        const = 200 # Параметр, который показывает насколько жадно работает скрипт. Без универа тянуло 275.
        s = defaultdict(list)
        r = requests.post('https://api.vk.com/method/execute.career?group_id='+
                          str(self.owner_id)+'&offset='+str(0)+'&v='+'5.73'+'&count='+str(const)
                          +'&access_token='+token).json()['response']
        
        members_count = r[0] #количество участников
        data = r[1] # наши данные
        print('В сообществе', self.owner_id , 'участников :', members_count)
        for k,v in data.items():
            s[k].extend(v)
        if members_count>const:
            for offset in range(const, members_count, const):
                try:
                    count = const
                    r = requests.post('https://api.vk.com/method/execute.career?group_id='+
                                     str(self.owner_id )+'&offset='+str(offset)+'&v='+'5.73'+'&count='+
                                     str(count)+'&access_token='+token).json()['response'][1]
                    for k,v in r.items():
                        s[k].extend(v)
                except Exception:
                    pass

        self.s = s
  

    def preparation(self):
        """Берет словарь из предыдущей части, создает датафрейм, 
        откуда удаляет строки, где не заполнено поле pos 
        и проводит с оставшимися строками различные манипуляции"""    
        df = pd.DataFrame.from_dict(self.s)
        df = df[pd.notnull(df['pos'])]
        df = df[list(self.s)].astype('str')
        df['age'] = year - pd.to_datetime(df['bdate'], errors = 'coerce').dt.year 
        df['link'] = 'https://vk.com/id' + df['ids'] 
        df['name'] = df['first_name'] +' '+ df['last_name']
        prepared = df.drop(['first_name', 'last_name', 'bdate'], axis = 1)
        self.prepared = prepared
        return self.prepared

    
    def sorting(self):
        """Принимает на вход отфильтрованный датасет, 
        проводит поиск по должностям и предлагает записать результат в csv
        В любом случае возвращает итоговый датасет"""
        all_data = pd.DataFrame()
        for pos in position:
            self.prepared[pos] = self.prepared['pos'].str.contains(pos, flags=re.IGNORECASE, regex=True)
            result = self.prepared[self.prepared[pos] == True]
            result.drop(pos, axis = 1, inplace = True)
            all_data = all_data.append(result, ignore_index=True)
        ########################################################
        all_data.drop_duplicates(subset = 'link', keep='first', inplace=True)
        all_data['group_name'] = requests.get('https://api.vk.com/method/groups.getById', 
            params = {'access_token':token, 
                      'v':'5.73', 'group_id':self.owner_id}).json()['response'][0]['name']
        sleep(0.33)
        all_data = all_data[['link', 'name',  'pos', 'com', 'faculty', 'age', 'city', 'group_name']]   
        return all_data
    
    
def main():
#     ids = get_by_txt(document_name) #если есть готовый txt
    ids = get_by_query() # если хотите получать данные по запросу
    position = str(input('Введите названия позиций через запятую: '))
    position = [x.strip() for x in position.split(',')]
    all_groups = pd.DataFrame()
    for group in ids:
        squeeze = Parcer(group).sorting()
        print('Подходящих по запросу: ', len(squeeze))
        all_groups = all_groups.append(squeeze, ignore_index=True)
    all_groups.drop_duplicates(subset = 'link', keep='first', inplace=True)
    excel_writer(all_groups)
    return all_groups


if __name__ == '__main__':
    main()



Введите поисковый запрос: smartdata
Введите названия позиций через запятую: java
В сообществе 147464741 участников : 524
Подходящих по запросу:  6
Хотите сохранить в эксель?[Y/N] n
Работа программы завершена
Wall time: 28.8 s


Параметры поля sort. Вдруг захочется поэксперементировать
* 0 — сортировать по умолчанию (аналогично результатам поиска в полной версии сайта);
* 1 — сортировать по скорости роста;
* 2 — сортировать по отношению дневной посещаемости к количеству пользователей;
* 3 — сортировать по отношению количества лайков к количеству пользователей;
* 4 — сортировать по отношению количества комментариев к количеству пользователей;
* 5 — сортировать по отношению количества записей в обсуждениях к количеству пользователей.
